In [22]:
import pandas as pd
from geopy.distance import geodesic
import numpy as np

In [24]:
geo = pd.read_csv('data/filtered_RJ_lat_long_data.csv')
final = pd.read_csv('data/final_combined_dataset.csv')

final.isnull().sum()

date                                    0
week                                    0
cases                                   0
population                              0
tempe_min                               0
humidity_max                            0
humidity_avg                            0
humidity_min                            0
temp_avg                                0
temp_max                                0
city                                    0
geocode                                 0
vim                                     0
vim_monthly                             0
precipitation_avg_ordinary_kriging      0
precipitation_max_ordinary_kriging      0
precipitation_avg_regression_kriging    0
precipitation_max_regression_kriging    0
dtype: int64

In [25]:
geo['CITY_lower'] = geo['CITY'].str.lower()
final['city_lower'] = final['city'].str.lower()

# Merge datasets using the lowercase city columns
final = pd.merge(final, geo[['CITY_lower', 'LONG', 'LAT']], left_on='city_lower', right_on='CITY_lower', how='left')

final = final.drop(columns=['CITY_lower', 'city_lower'])

# rename 
final = final.rename(columns={'LONG': 'long', 'LAT': 'lat'})


In [26]:
final['cases_per_100k'] = final['cases'] / final['population'] * 100000

Spatial Aggregation: Create additional features that represent the cases of nearby cities.
For example:
Average cases of neighboring cities.

nearby_cases_weighted: Weighted average of cases in the 3 nearest cities for the same week.

here we will consider cases_per_100k as cases. Because we want to consider the population of the city in the calculation.

In [27]:
final.to_csv('data/final_combined_dataset.csv', index=False)

In [28]:
data = pd.read_csv('data/final_combined_dataset.csv')


def calculate_weighted_cases(row, weekly_data, top_k=3):
    base_coords = (row['lat'], row['long'])
    distances = []

    # Iterate through other cities in the same week
    for _, city_row in weekly_data.iterrows():
        if city_row['city'] != row['city']:
            # Calculate distance between cities
            dist = geodesic(base_coords, (city_row['lat'], city_row['long'])).kilometers
            distances.append((city_row['city'], city_row['cases_per_100k'], dist))

    # Sort by distance and take the top_k nearest cities
    nearest = sorted(distances, key=lambda x: x[2])[:top_k]

    # Compute weighted cases: sum(weighted cases) / sum(weights)
    weighted_cases = sum(city[1] / city[2] for city in nearest) / sum(1 / city[2] for city in nearest) if nearest else 0

    return weighted_cases

In [29]:
# First part: Weeks from 201201 to 201701
first_part_weeks = list(range(201201, 201701))

# Update for the first part
first_part_data = data[data['week'].isin(first_part_weeks)]
nearby_weighted_cases_first_part = []

for week in first_part_data['week'].unique():
    weekly_data = first_part_data[first_part_data['week'] == week]
    print(f'Processing week {week}')
    for _, row in weekly_data.iterrows():
        weighted_cases = calculate_weighted_cases(row, weekly_data)
        nearby_weighted_cases_first_part.append(weighted_cases)

data.loc[data['week'].isin(first_part_weeks), 'nearby_cases_weighted'] = nearby_weighted_cases_first_part

# Save the updated data back to CSV
data.to_csv('data/final_combined_dataset.csv', index=False)


Processing week 201201
Processing week 201202
Processing week 201203
Processing week 201204
Processing week 201205
Processing week 201206
Processing week 201207
Processing week 201208
Processing week 201209
Processing week 201210
Processing week 201211
Processing week 201212
Processing week 201213
Processing week 201214
Processing week 201215
Processing week 201216
Processing week 201217
Processing week 201218
Processing week 201219
Processing week 201220
Processing week 201221
Processing week 201222
Processing week 201223
Processing week 201224
Processing week 201225
Processing week 201226
Processing week 201227
Processing week 201228
Processing week 201229
Processing week 201230
Processing week 201231
Processing week 201232
Processing week 201233
Processing week 201234
Processing week 201235
Processing week 201236
Processing week 201237
Processing week 201238
Processing week 201239
Processing week 201240
Processing week 201241
Processing week 201242
Processing week 201243
Processing 

In [30]:
data.head()

,date,week,cases,population,tempe_min,humidity_max,humidity_avg,humidity_min,temp_avg,temp_max,...,vim,vim_monthly,precipitation_avg_ordinary_kriging,precipitation_max_ordinary_kriging,precipitation_avg_regression_kriging,precipitation_max_regression_kriging,long,lat,cases_per_100k,nearby_cases_weighted
0,2012-01-01,201201,32,207044,19.000000,79.428571,55.514486,35.000000,25.048951,29.571429,...,0.850500,0.852633,5.6804,18.3375,5.2038,18.7979,-44.319627,-23.009116,15.455652,6.913995
1,2012-01-08,201202,40,207044,19.714286,82.285714,62.357393,47.428571,23.737513,26.571429,...,0.852050,0.852633,4.0716,16.5375,3.9114,16.8583,-44.319627,-23.009116,19.319565,32.825012
2,2012-01-15,201203,19,207044,20.000000,83.000000,65.236264,45.571429,24.413187,28.714286,...,0.853541,0.852633,2.0555,5.0500,1.7556,3.1368,-44.319627,-23.009116,9.176793,14.685214
3,2012-01-22,201204,33,207044,19.285714,83.000000,60.362637,43.428571,24.879121,28.857143,...,0.854877,0.852633,1.5416,5.7059,1.4138,5.2767,-44.319627,-23.009116,15.938641,2.809255
4,2012-01-29,201205,36,207044,18.857143,80.857143,50.885924,33.142857,25.989992,30.428571,...,0.856021,0.852633,2.8204,9.3826,2.0057,7.8749,-44.319627,-23.009116,17.387608,19.938997


In [32]:
# Second part: Weeks from 201702 onwards
second_part_weeks = list(range(201701, 202301))

# Update for the second part
second_part_data = data[data['week'].isin(second_part_weeks)]
nearby_weighted_cases_second_part = []

for week in second_part_data['week'].unique():
    weekly_data = second_part_data[second_part_data['week'] == week]
    print(f'Processing week {week}')
    for _, row in weekly_data.iterrows():
        weighted_cases = calculate_weighted_cases(row, weekly_data)
        nearby_weighted_cases_second_part.append(weighted_cases)

data.loc[data['week'].isin(second_part_weeks), 'nearby_cases_weighted'] = nearby_weighted_cases_second_part

# Save the updated data back to CSV
data.to_csv('data/final_combined_dataset.csv', index=False)


Processing week 201701
Processing week 201702
Processing week 201703
Processing week 201704
Processing week 201705
Processing week 201706
Processing week 201707
Processing week 201708
Processing week 201709
Processing week 201710
Processing week 201711
Processing week 201712
Processing week 201713
Processing week 201714
Processing week 201715
Processing week 201716
Processing week 201717
Processing week 201718
Processing week 201719
Processing week 201720
Processing week 201721
Processing week 201722
Processing week 201723
Processing week 201724
Processing week 201725
Processing week 201726
Processing week 201727
Processing week 201728
Processing week 201729
Processing week 201730
Processing week 201731
Processing week 201732
Processing week 201733
Processing week 201734
Processing week 201735
Processing week 201736
Processing week 201737
Processing week 201738
Processing week 201739
Processing week 201740
Processing week 201741
Processing week 201742
Processing week 201743
Processing 